In [1]:
from os import listdir
from clustimage import Clustimage
import matplotlib.pyplot as plt
from PIL import Image as PImage
import io
import imagehash
from undouble import Undouble


<frozen importlib._bootstrap>:219: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [2]:
def loadImages(path):
    # return array of images

    imagesList = listdir(path)
    loadedImages = []
    for image in imagesList:
        img = PImage.open(path + image)
        loadedImages.append(img)

    return loadedImages

path = "/Users/vidyakumar/Desktop/ds/python/gst_image_analysis/image_sample/"

# your images in an array
imgs = loadImages(path)

In [3]:
for img in imgs:
    # you can show every image
    img.show()

In [4]:
# prints format of image
print(imgs[0].format)
 
# prints mode of image
print(imgs[0].mode)

JPEG
RGB


In [5]:
print(imgs[0])

<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=1024x768 at 0x10DE15EB0>


In [8]:
# Initialize
cl = Clustimage()

# Preprocessing of the first image
# 0: cv2.IMREAD_GRAYSCALE
# 1: cv2.IMREAD_COLOR
img_1 = cl.imread("/ds/python/gst_image_analysis/image_sample/"+listdir(path)[0], dim=(128,128), colorscale=0, flatten=True)

# Flattened array
print(img_1)
# array([ 10,  22,  13, ...,  78,  74, 117], dtype=uint8)

[undouble] >INFO> filepath is set to [/var/folders/k8/853v_n2j4mg1fczdrs9bspsh0000gn/T/clustimage]
[undouble] >INFO> filepath is set to [/var/folders/k8/853v_n2j4mg1fczdrs9bspsh0000gn/T/clustimage]
[undouble] >INFO> filepath is set to [/var/folders/k8/853v_n2j4mg1fczdrs9bspsh0000gn/T/clustimage]
[undouble] >WARNING> Scaling not possible.
[undouble] >WARNING> Could not read: [/ds/python/gst_image_analysis/image_sample/002406.jpg]


None


In [7]:
img_1.shape

AttributeError: 'NoneType' object has no attribute 'shape'

In [ ]:
# Plot. Note that reshape is only required in case flatten=True
plt.figure()
plt.imshow(img_1.reshape(128,128))
plt.axis('off')

In [ ]:
img_2 = cl.imread("/Users/vidyakumar/Desktop/ds/python/gst_image_analysis/image_sample/"+listdir(path)[1], dim=(128,128), colorscale=0, flatten=True)
print(img_2)

In [ ]:
img_2.shape

In [ ]:
# Plot. Note that reshape is only required in case flatten=True
plt.figure()
plt.imshow(img_2.reshape(128,128))
plt.axis('off')

In [ ]:
model = Undouble(method='phash', hash_size=16)

In [ ]:
model.import_data(path)

In [ ]:
model.compute_hash()

In [ ]:
model.group(threshold=0)

In [ ]:
model.plot()

In [ ]:
#https://towardsdatascience.com/detection-of-duplicate-images-using-image-hash-functions-4d9c53f04a75
#https://towardsdatascience.com/a-step-by-step-guide-for-clustering-images-4b45f9906128

In [ ]:
#Results
#Took a sample of 2 images and created a copy of both - total 4 images
#2 groups were created - same images grouped

In [1]:
from sentence_transformers import SentenceTransformer, util
from PIL import Image
import glob
import os

<frozen importlib._bootstrap>:219: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [2]:
# Load the OpenAI CLIP Model
print('Loading CLIP Model...')
model = SentenceTransformer('clip-ViT-B-32')

Loading CLIP Model...


In [3]:
filepath = "/Users/vidyakumar/Desktop/ds/python/gst_image_analysis/images/"

# Next we compute the embeddings
# To encode an image, you can use the following code:
# from PIL import Image
# encoded_image = model.encode(Image.open(filepath))
image_names = list(glob.glob(filepath + '*'))
print("Images:", len(image_names))
encoded_image = model.encode([Image.open(filepath) for filepath in image_names], batch_size=128, convert_to_tensor=True, show_progress_bar=True)

Images: 2407


Batches:   0%|          | 0/19 [00:00<?, ?it/s]

In [6]:
filepath1 = "/Users/vidyakumar/Desktop/ds/python/gst_image_analysis/image_sample/"

# Next we compute the embeddings
# To encode an image, you can use the following code:
# from PIL import Image
# encoded_image = model.encode(Image.open(filepath))
image_names1 = list(glob.glob(filepath1 + '*'))
print("Images:", len(image_names1))
encoded_image1 = model.encode([Image.open(filepath1) for filepath1 in image_names1], batch_size=128, convert_to_tensor=True, show_progress_bar=True)

Images: 1


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [43]:
# Now we run the clustering algorithm. This function compares images aganist 
# all other images and returns a list with the pairs that have the highest 
# cosine similarity score
processed_images = util.paraphrase_mining_embeddings(encoded_image)
NUM_SIMILAR_IMAGES = 20 

In [44]:
print('Finding duplicate images...')
# Filter list for duplicates. Results are triplets (score, image_id1, image_id2) and is scorted in decreasing order
# A duplicate image will have a score of 1.00
duplicates = [image for image in processed_images if image[0] >= 1]

# Output the top X duplicate images
for score, image_id1, image_id2 in duplicates[0:NUM_SIMILAR_IMAGES]:
    print("\nScore: {:.3f}%".format(score * 100))
    print(image_names[image_id1])
    print(image_names[image_id2])

Finding duplicate images...

Score: 100.000%
/Users/vidyakumar/Desktop/ds/python/gst_image_analysis/images/002284.jpg
/Users/vidyakumar/Desktop/ds/python/gst_image_analysis/images/000855.jpg


In [45]:
print('Finding near duplicate images...')
# Use a threshold parameter to identify two images as similar. By setting the threshold lower, 
# you will get larger clusters which have less similar images in it. Threshold 0 - 1.00
# A threshold of 1.00 means the two images are exactly the same. Since we are finding near 
# duplicate images, we can set it at 0.99 or any number 0 < X < 1.00.
threshold = 0.99
near_duplicates = [image for image in processed_images if image[0] < threshold]

for score, image_id1, image_id2 in near_duplicates[0:NUM_SIMILAR_IMAGES]:
    print("\nScore: {:.3f}%".format(score * 100))
    print(image_names[image_id1])
    print(image_names[image_id2])

Finding near duplicate images...

Score: 97.748%
/Users/vidyakumar/Desktop/ds/python/gst_image_analysis/images/001501.jpg
/Users/vidyakumar/Desktop/ds/python/gst_image_analysis/images/001500.jpg

Score: 97.610%
/Users/vidyakumar/Desktop/ds/python/gst_image_analysis/images/001886.jpg
/Users/vidyakumar/Desktop/ds/python/gst_image_analysis/images/000659.jpg

Score: 97.384%
/Users/vidyakumar/Desktop/ds/python/gst_image_analysis/images/000444.jpg
/Users/vidyakumar/Desktop/ds/python/gst_image_analysis/images/000405.jpg

Score: 97.362%
/Users/vidyakumar/Desktop/ds/python/gst_image_analysis/images/001703.jpg
/Users/vidyakumar/Desktop/ds/python/gst_image_analysis/images/001702.jpg

Score: 97.340%
/Users/vidyakumar/Desktop/ds/python/gst_image_analysis/images/002386.jpg
/Users/vidyakumar/Desktop/ds/python/gst_image_analysis/images/000284.jpg

Score: 97.269%
/Users/vidyakumar/Desktop/ds/python/gst_image_analysis/images/001175.jpg
/Users/vidyakumar/Desktop/ds/python/gst_image_analysis/images/001226

In [ ]:
#changing orientation 98%
#changing orientation&resolution 97%
#changing orientation&resolution&size 96%
#changing colors&other effect 85%